<a href="https://colab.research.google.com/github/loraynegulbr/qtm151spring2025/blob/main/Nursing_Data_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from google.colab import data_table
data_table.DataTable.max_columns = 100

censuscounties = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/censuscounties.csv')

# filter yr 6
censuscountiesyr = censuscounties[~censuscounties['YEAR'].isin([1, 2, 3, 4, 5])]

# drop extra columns
censuscountiesabv = censuscountiesyr[
    ['COUNTY', 'CTYNAME', 'TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'H_MALE', 'H_FEMALE']
    ].copy()

# combine men + women
import re

male_cols = [col for col in censuscountiesabv.columns if col.endswith('_MALE')]
female_cols = [col for col in censuscountiesabv.columns if col.endswith('_FEMALE')]

male_prefixes = {col[:-5] for col in male_cols}     # remove _MALE
female_prefixes = {col[:-7] for col in female_cols} # remove _FEMALE

prefixes = male_prefixes & female_prefixes          # intersection: real pairs

for prefix in prefixes:
    censuscountiesabv[f'{prefix}_TOTAL'] = censuscountiesabv[f'{prefix}_MALE'] + censuscountiesabv[f'{prefix}_FEMALE']

# drop individual gender pop's
censuscountiestotals = censuscountiesabv[
    ['COUNTY', 'CTYNAME', 'TOT_POP', 'WA_TOTAL', 'BA_TOTAL', 'IA_TOTAL', 'AA_TOTAL', 'NA_TOTAL', 'TOM_TOTAL', 'H_TOTAL']
    ]

# sum pop's by county
censuscountiesclean = censuscountiestotals.groupby(['COUNTY', 'CTYNAME'], as_index=False).sum()

# FIPS
censuscountiesclean['FIPS'] = censuscountiesclean['COUNTY'].apply(lambda x: f"13{int(x):03d}")



censuscountiesclean

,COUNTY,CTYNAME,TOT_POP,WA_TOTAL,BA_TOTAL,IA_TOTAL,AA_TOTAL,NA_TOTAL,TOM_TOTAL,H_TOTAL,FIPS
0,1,Appling County,37338,28502,7236,360,366,94,780,4346,13001
1,3,Atkinson County,16870,12708,2742,344,200,310,566,4998,13003
2,5,Bacon County,22090,17294,3878,108,262,58,490,1986,13005
3,7,Baker County,5464,2944,2222,82,100,4,112,372,13007
4,9,Baldwin County,87288,47100,36846,312,1524,76,1430,2694,13009
...,...,...,...,...,...,...,...,...,...,...,...
154,313,Whitfield County,210140,188052,9524,4126,4074,520,3844,80792,13313
155,315,Wilcox County,17808,10986,6224,58,168,6,366,726,13315
156,317,Wilkes County,19134,10786,7500,58,222,12,556,1006,13317
157,319,Wilkinson County,17334,10370,6370,130,94,4,366,636,13319


In [9]:
import plotly.express as px

censuscountiesclean['hover_text'] = censuscountiesclean.apply(lambda row:
                                        f"{row['CTYNAME']}<br>"
                                        f"Total: {row['TOT_POP']}<br>"
                                        f"White: {row['WA_TOTAL']}<br>"
                                        f"AfrAmer: {row['BA_TOTAL']}<br>"
                                        f"AmerIndian: {row['IA_TOTAL']}<br>"
                                        f"Asian: {row['AA_TOTAL']}<br>"
                                        f"NatHawaiian: {row['NA_TOTAL']}<br>"
                                        f"Biracial: {row['TOM_TOTAL']}<br>"
                                        f"Hispanic: {row['H_TOTAL']}", axis=1
                                        )

fig = px.choropleth(
    censuscountiesclean,
    geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
    locations='FIPS',       # column with FIPS codes
    color='WA_TOTAL',          # can use any column for color
    hover_name='CTYNAME',
    hover_data={'FIPS': False},
    scope='usa',            # zooms to USA
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(hovertemplate=censuscountiesclean['hover_text'])

# Add our custom hover text
fig.update_traces(hovertemplate=censuscountiesclean['hover_text'])

# Focus on Georgia
fig.update_geos(fitbounds="locations", visible=False)

fig.show()